In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [2]:

!pip install pyspark

     |████████████████████████████████| 217.8MB 61kB/s 
     |████████████████████████████████| 204kB 47.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=faaf446c0201072fede5fa767f3eb748e8e9f6ea6bb66f06152c46bb23068f16
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [3]:
!git clone https://github.com/leonardoamorim/aulapython.git

Cloning into 'aulapython'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 25 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [0]:
! gunzip aulapython/NASA_access_log_Jul95.gz
! gunzip aulapython/NASA_access_log_Aug95.gz

In [0]:
from pyspark import SparkConf, SparkContext
from operator import add

In [0]:
configuracao = (SparkConf().setMaster("local").setAppName("Solucao")
                .set("spark.executor.memory", "5g"))

In [0]:
sc = SparkContext(conf = configuracao)

In [0]:
julho = sc.textFile('aulapython/NASA_access_log_Jul95')
agosto = sc.textFile('aulapython/NASA_access_log_Aug95')

In [0]:
julho = julho.cache()
agosto = agosto.cache()

In [10]:
# Numero de hosts distintos no mes de Julho
contagem_julho = julho.map(lambda line: line.split(' ')[0]).distinct().count()
print("Numero de hosts distintos no mes de Julho:", contagem_julho)

Numero de hosts distintos no mes de Julho: 81983


In [11]:
# Numero de hosts distintos no mes de Agosto
contagem_agosto = agosto.map(lambda line: line.split(' ')[0]).distinct().count()
print("Numero de hosts distintos no mes de Agosto:", contagem_agosto)

Numero de hosts distintos no mes de Agosto: 75060


In [0]:
# Funcao para verificar se em uma linha existe um codigohttp igual a 404
def codigo404(linha):
    try:
        codigohttp = linha.split(' ')[-2]
        if codigohttp == '404':
            return True
    except:
        pass
    return False

In [0]:
erros404_julho = julho.filter(codigo404).cache()
erros404_agosto = agosto.filter(lambda linha: linha.split(' ')[-2] == '404').cache()

In [14]:
print('Erros 404 em Julho: %s' % erros404_julho.count())

Erros 404 em Julho: 10845


In [15]:
print('Erros 404 em Agosto: %s' % erros404_agosto.count())

Erros 404 em Agosto: 10056


In [0]:
# Funcao para retornar os top5 urls com mais erros 404
def top5_hosts404(rdd):
    hosts = rdd.map(lambda linha: linha.split('"')[1].split(' ')[1])
    counts = hosts.map(lambda hosts: (hosts, 1)).reduceByKey(add)
    top5 = counts.sortBy(lambda par: -par[1]).take(5)
    
    print('\nTop 5 urls:')
    for hosts, count in top5:
        print(hosts, count)
        
    return top5

In [17]:
top5_hosts404(erros404_julho)


Top 5 urls:
/pub/winvn/readme.txt 667
/pub/winvn/release.txt 547
/history/apollo/apollo-13.html 286
/shuttle/resources/orbiters/atlantis.gif 232
/history/apollo/a-001/a-001-patch-small.gif 230


[('/pub/winvn/readme.txt', 667),
 ('/pub/winvn/release.txt', 547),
 ('/history/apollo/apollo-13.html', 286),
 ('/shuttle/resources/orbiters/atlantis.gif', 232),
 ('/history/apollo/a-001/a-001-patch-small.gif', 230)]

In [18]:
top5_hosts404(erros404_agosto)


Top 5 urls:
/pub/winvn/readme.txt 1337
/pub/winvn/release.txt 1185
/shuttle/missions/STS-69/mission-STS-69.html 683
/images/nasa-logo.gif 319
/shuttle/missions/sts-68/ksc-upclose.gif 253


[('/pub/winvn/readme.txt', 1337),
 ('/pub/winvn/release.txt', 1185),
 ('/shuttle/missions/STS-69/mission-STS-69.html', 683),
 ('/images/nasa-logo.gif', 319),
 ('/shuttle/missions/sts-68/ksc-upclose.gif', 253)]

In [0]:
# Quantidade de erros 404 por dia
def contador_dias_404(rdd):
    dias = rdd.map(lambda linha: linha.split('[')[1].split(':')[0])
    counts = dias.map(lambda dia: (dia, 1)).reduceByKey(add).collect()
    return counts

In [20]:
contador_dias_404(erros404_julho)

[('13/Jul/1995', 532),
 ('21/Jul/1995', 334),
 ('25/Jul/1995', 461),
 ('09/Jul/1995', 348),
 ('15/Jul/1995', 254),
 ('16/Jul/1995', 257),
 ('18/Jul/1995', 465),
 ('17/Jul/1995', 406),
 ('07/Jul/1995', 570),
 ('12/Jul/1995', 471),
 ('19/Jul/1995', 639),
 ('22/Jul/1995', 192),
 ('23/Jul/1995', 233),
 ('03/Jul/1995', 474),
 ('05/Jul/1995', 497),
 ('10/Jul/1995', 398),
 ('14/Jul/1995', 413),
 ('01/Jul/1995', 316),
 ('02/Jul/1995', 291),
 ('04/Jul/1995', 359),
 ('06/Jul/1995', 640),
 ('08/Jul/1995', 302),
 ('11/Jul/1995', 471),
 ('20/Jul/1995', 428),
 ('24/Jul/1995', 328),
 ('26/Jul/1995', 336),
 ('27/Jul/1995', 336),
 ('28/Jul/1995', 94)]

In [21]:
contador_dias_404(erros404_agosto)

[('01/Aug/1995', 243),
 ('07/Aug/1995', 537),
 ('09/Aug/1995', 279),
 ('10/Aug/1995', 315),
 ('21/Aug/1995', 305),
 ('27/Aug/1995', 370),
 ('30/Aug/1995', 571),
 ('03/Aug/1995', 304),
 ('06/Aug/1995', 373),
 ('08/Aug/1995', 391),
 ('16/Aug/1995', 259),
 ('20/Aug/1995', 312),
 ('05/Aug/1995', 236),
 ('11/Aug/1995', 263),
 ('12/Aug/1995', 196),
 ('13/Aug/1995', 216),
 ('15/Aug/1995', 327),
 ('17/Aug/1995', 271),
 ('22/Aug/1995', 288),
 ('23/Aug/1995', 345),
 ('24/Aug/1995', 420),
 ('26/Aug/1995', 366),
 ('28/Aug/1995', 410),
 ('14/Aug/1995', 287),
 ('18/Aug/1995', 256),
 ('19/Aug/1995', 209),
 ('31/Aug/1995', 526),
 ('04/Aug/1995', 346),
 ('25/Aug/1995', 415),
 ('29/Aug/1995', 420)]

In [0]:
# Quantidade de bytes acumulados 
def quantidade_bytes_acumulados(rdd):
    def contador(linha):
        try:
            count = int(linha.split(" ")[-1])
            if count < 0:
                raise ValueError()
            return count
        except:
            return 0
        
    count = rdd.map(contador).reduce(add)
    return count

In [23]:
print('Quantidade de bytes total em Julho: %s' % quantidade_bytes_acumulados(julho))

Quantidade de bytes total em Julho: 38695973491


In [24]:
print('Quantidade de bytes total em Agosto: %s' % quantidade_bytes_acumulados(agosto))

Quantidade de bytes total em Agosto: 26828341424


In [0]:
sc.stop()